# Descargar datos y dependencias

In [1]:
import numpy as np
import json
import math
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import time

In [2]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2025-05-31 15:25:43--  https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0 [following]
--2025-05-31 15:25:43--  https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc480552ea31971949d9b45744f9.dl.dropboxusercontent.com/cd/0/inline/CqtAtCYGmLRkwCfjK3-IIFWvIRzQaWs2Lg08nzOVR81WRm4oJm-643Fs4xgbBhFJuTYXHx0jM4SBGNAWOYgDXBvwwH2m8BqirxlKeX4UqckezDxApSSpzmlPx4j-xZFwfI-6tPBfStiFZk7arhiySVL8/file# [following]
--2025-05-31 15:25:44--  https://uc480552ea31971949d9b45744f9.dl.dropboxusercontent.com/cd/0/in

In [3]:
!pip install lightfm

In [4]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
import numpy as np
import pandas as pd
import json
from scipy.sparse import coo_matrix, csr_matrix
import scipy.sparse


# Cargar Datos

In [5]:
df_books = pd.read_csv('books.csv', sep=',')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [6]:
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

In [7]:
with open("goodreads_past_interactions.json", "r") as f:
    user_interactions = json.load(f)

In [8]:
with open("goodreads_test_interactions.json", "r") as f:
    user_interactions_test = json.load(f)

In [9]:
rows = []
cols = []
data = []

for user_id, book_ids in user_interactions.items():
    for book_id in book_ids:
        rows.append(int(user_id))
        cols.append(int(book_id))
        data.append(1)

max_user_id = max(rows)
max_item_id = max(cols)

# Crear matriz sparse
user_item_matrix_train = coo_matrix((data, (rows, cols)), shape=(max_user_id + 1, max_item_id + 1))

In [10]:
rows_test = []
cols_test = []
data_test = []

for user_id, book_ids in user_interactions_test.items():
    for book_id in book_ids:
        rows_test.append(int(user_id))
        cols_test.append(int(book_id))
        data_test.append(1)

max_user_id = max([max_user_id] + rows_test)
max_item_id = max([max_item_id] + cols_test)

# Crear matriz sparse
user_item_matrix_test = coo_matrix((data, (rows, cols)), shape=(max_user_id + 1, max_item_id + 1))

In [ ]:
# 5. Entrenar modelo LightFM
model = LightFM(loss='warp')  # o 'bpr', 'warp-kos', 'logistic'
model.fit(user_item_matrix_train, epochs=20, num_threads=4)

In [ ]:
# 6. Evaluar
print("Precision@10 (test):", precision_at_k(model, user_item_matrix_test, k=10).mean())
print("AUC (test):", auc_score(model, user_item_matrix_test).mean())

BERT Embeddings

In [11]:
bert_embeddings = np.load("goodreads_bert_embeddings.npy")  # shape: (num_items, embedding_dim)
bert_embeddings_large = np.load("goodreads_bert_large_embeddings.npy")
num_items, embedding_dim = bert_embeddings.shape
num_items, embedding_dim_large = bert_embeddings.shape

print(bert_embeddings.shape)
print(bert_embeddings_large.shape)

(4287, 768)
(4287, 1024)


In [12]:
bert_embeddings

array([[ 0.23684844, -0.09462399, -0.01145189, ...,  0.12350288,
        -0.18344034,  0.45543516],
       [ 0.3566235 ,  0.02383685,  0.09952748, ...,  0.16955282,
        -0.00520906,  0.08447403],
       [ 0.22800492,  0.09910604, -0.04040277, ...,  0.19354148,
        -0.0337148 ,  0.27135998],
       ...,
       [ 0.20124024, -0.09574547,  0.07568537, ...,  0.21734682,
         0.03540741,  0.32540751],
       [ 0.15547206, -0.10936089,  0.23241168, ...,  0.0934333 ,
        -0.12637904,  0.1302035 ],
       [ 0.38171366, -0.01875851,  0.04588691, ...,  0.26124743,
         0.08366313,  0.26039878]])

In [13]:
from sklearn.preprocessing import normalize

bert_embeddings = normalize(bert_embeddings, norm='l2')

In [ ]:
item_features_sparse = csr_matrix(bert_embeddings)


In [ ]:
user_item_matrix_test

In [ ]:
user_item_matrix_train.shape

In [ ]:
user_item_matrix_train_csr = user_item_matrix_train.tocsr()
user_item_matrix_test_csr = user_item_matrix_test.tocsr()

In [ ]:
user_item_matrix_train_csr.shape

Normalizar es opcional.

Por qué normalizar embeddings?
Facilita el entrenamiento al evitar que ciertos ítems "dominen" por tener valores grandes.

Es especialmente útil si entrenas con dot-product o coseno, como ocurre con LightFM y muchos sistemas de recomendación.

Puede mejorar la estabilidad y la velocidad de convergencia.

No es obligatorio, pero generalmente recomendado si usas embeddings como features.

In [ ]:
model_bert = LightFM(loss='warp')
model_bert.fit(interactions=user_item_matrix_train, item_features=item_features_sparse, epochs=20, num_threads=4)


In [ ]:
from lightfm.evaluation import precision_at_k, auc_score

# Evaluación en test
precision_at_k(model, user_item_matrix_test, item_features=item_features_sparse, k=10).mean()
auc_score(model, user_item_matrix_test, item_features=item_features_sparse).mean()

In [14]:
import numpy as np
import scipy.sparse

# Paso 1: Cargar los embeddings
raw_embeddings = np.load("goodreads_bert_embeddings.npy")  # shape (num_books_with_embeddings, embedding_dim)

# Paso 2: Preparar matriz vacía con forma (10000, embedding_dim)
n_items = user_item_matrix_train.shape[1]
embedding_dim = raw_embeddings.shape[1]
bert_embeddings_aligned = np.zeros((n_items, embedding_dim))

# Paso 3: Rellenar embeddings alineados según book_id → index
for book_id, idx_in_embeddings in bookid2idx.items():
    if book_id < n_items:  # Solo si book_id está dentro de los IDs usados en la matriz
        bert_embeddings_aligned[book_id] = raw_embeddings[idx_in_embeddings]

# Paso 4: Convertir a matriz sparse
item_features_sparse = csr_matrix(bert_embeddings_aligned)


In [15]:
bert_embeddings_aligned.shape

(10000, 768)

In [16]:
item_features_sparse.shape

(10000, 768)

Revisando zero-values, ya que no hay embeddings para todos los libros dentro de matriz. (tal vez revisar si sacarlos del training set).

In [17]:
zero_rows = np.sum(np.all(bert_embeddings_aligned == 0, axis=1))
print(f"Items with no embedding: {zero_rows} out of {bert_embeddings_aligned.shape[0]}")

Items with no embedding: 5713 out of 10000


Filtrar items sin embeddings BERT

In [27]:
import numpy as np

zero_row_mask = np.all(bert_embeddings_aligned == 0, axis=1)
nonzero_item_indices = np.where(~zero_row_mask)[0]  # índices de ítems validos

In [28]:
filtered_embeddings = bert_embeddings_aligned[nonzero_item_indices]
item_features_sparse = scipy.sparse.csr_matrix(filtered_embeddings)

In [29]:
item_features_sparse.shape

(4287, 768)

In [30]:
user_item_matrix_train_csr = user_item_matrix_train.tocsr()
user_item_matrix_test_csr = user_item_matrix_test.tocsr()

# Ahora puedes indexar por columnas
user_item_matrix_train_filtered = user_item_matrix_train_csr[:, nonzero_item_indices]
user_item_matrix_test_filtered = user_item_matrix_test_csr[:, nonzero_item_indices]

In [31]:
user_item_matrix_train_filtered.shape

(53425, 4287)

In [33]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score

model = LightFM(loss='warp')
model.fit(user_item_matrix_train_filtered, item_features=item_features_sparse, epochs=5, num_threads=4)

In [34]:
# Evaluar en entrenamiento
print("Precision@10:", precision_at_k(model, user_item_matrix_test_filtered, item_features=item_features_sparse, k=10).mean())


Precision@10: 0.34815505
